# **Using The Phi3 SML Locally In Microsoft Fabric For GenAI**

### Sandeep Pawar  |   fabric.guru  |  May 21, 2024

In [1]:
#install onnx runtime
!pip install --pre onnxruntime-genai --q

StatementMeta(, d0e70469-2ecd-45f7-8c14-bfbd81b0a2fc, 3, Finished, Available)

In [2]:
#create a directory for the model
import os 
import shutil
import onnxruntime_genai as og

model_path = '/lakehouse/default/Files/phi3mini'

#Mount a lakehouse first
if not os.path.exists(model_path):
    os.mkdir(model_path)
    print(f"model will be downloaded to {model_path}")
else:
    print(f"{model_path} exists")


StatementMeta(, d0e70469-2ecd-45f7-8c14-bfbd81b0a2fc, 4, Finished, Available)

/lakehouse/default/Files/phi3mini exists


In [3]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-onnx --include cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/* --local-dir ./lakehouse/default/Files/phi3mini

StatementMeta(, d0e70469-2ecd-45f7-8c14-bfbd81b0a2fc, 5, Finished, Available)

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Fetching 10 files:   0%|                                 | 0/10 [00:00<?, ?it/s]downloading https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-onnx/resolve/24fd626412942b0bcd8f16393ef10b69cfc2d162/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/added_tokens.json to /home/trusted-service-user/.cache/huggingface/hub/tmp0gfaqnud
downloading https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-onnx/resolve/24fd626412942b0bcd8f16393ef10b69cfc2d162/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/genai_config.json to /home/trusted-service-user/.cache/huggingface/hub/tmp3cv861g2
downloading https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-onnx/resolve/24fd626412942b0bcd8f16393ef10b69cfc2d162/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/config.json to /home/trusted-service-user/.cache/huggingface/hub/tmpa

In [4]:
#copy files to a lakehouse from the temp directory
source_dir = os.path.abspath(".") + model_path
destination_dir = model_path

try:
    shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)
    print("Directory copied successfully.")
except Exception as e:
    print(f"Error occurred: {e}")


StatementMeta(, d0e70469-2ecd-45f7-8c14-bfbd81b0a2fc, 6, Finished, Available)

Directory copied successfully.


In [5]:
#confirm files have been copied
for f in mssparkutils.fs.ls('Files/phi3mini/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4'):
    print(f.name)

StatementMeta(, d0e70469-2ecd-45f7-8c14-bfbd81b0a2fc, 7, Finished, Available)

added_tokens.json
config.json
configuration_phi3.py
genai_config.json
phi3-mini-4k-instruct-cpu-int4-rtn-block-32-acc-level-4.onnx
phi3-mini-4k-instruct-cpu-int4-rtn-block-32-acc-level-4.onnx.data
special_tokens_map.json
tokenizer.json
tokenizer.model
tokenizer_config.json


In [7]:
#initialize the model
model = og.Model(model_path+'/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4')

#tokenize
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

#set params
search_options = {"max_length": 1024,"temperature":0.6}
params = og.GeneratorParams(model)
params.try_graph_capture_with_max_batch_size(1)
params.set_search_options(**search_options)

#provide instructions and prompts
instruction = "You are a renowed poet known for his playful poems"
prompt = " Write a poem to convince someone why pineapple on pizza is awesome"
prompt = f"<|system|>{instruction}<|end|><|user|>{prompt}<|end|><|assistant|>"

#tokenize prompt
input_tokens = tokenizer.encode(prompt)
params.input_ids = input_tokens

#generate response
generator = og.Generator(model, params)
while not generator.is_done():
                generator.compute_logits()
                generator.generate_next_token()

                new_token = generator.get_next_tokens()[0]
                print(tokenizer_stream.decode(new_token), end='', flush=True)

StatementMeta(, d0e70469-2ecd-45f7-8c14-bfbd81b0a2fc, 9, Finished, Available)

 In a land where flavors collide,
A culinary adventure, side by side,
A tale of tastes, a sweet and savory ride,
A pineapple on pizza, a delightful stride.

A golden crown of fruit, a tropical sight,
Glistening under the kitchen's warm light,
A slice of paradise, a flavor so bright,
Pineapple on pizza, a taste bud' fearsome fight.

Sweet and tangy, a flavor explosion,
A symphony of tastes, a flavor fusion,
A pizza slice, a pineapple's illusion,
A culinary masterpiece, a gastronomic revolution.

Crispy crust, a foundation so firm,
A bed for the fruit, a taste to affirm,
A pineapple's sweetness, a flavor to confirm,
A pizza slice, a culinary term.

Skeptics may scoff, critics may jeer,
But taste buds will sing, and hearts will cheer,
For the pineapple on pizza, a flavor so clear,
A delicious delight, a culinary pioneer.

So here's to the pineapple, a fruit so bold,
On a pizza slice, a story untold,
A taste so unique, a flavor to behold,
Pineapple on pizza, a culinary gold.

So come, take